In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
hotel_list = []
for dirname, _, filenames in os.walk('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images'):
    for filename in filenames:
        hotel_list.append(os.path.join(dirname, filename))
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os
import numpy as np
import pandas as pd
import tqdm
import cv2
import matplotlib.pyplot as plt

from joblib import Parallel, delayed

## Reading in the csv we created during EDA
This csv contains the image to hotel mapping

In [7]:
df = pd.read_csv('/kaggle/input/image2hotelmapping/image_to_hotel_mapping.csv')
# df['image_id']

In [8]:
df['hotel_id'].sample(5)

15012      5558
9334      23860
17975    308904
26355     28105
34525     33486
Name: hotel_id, dtype: int64

In [9]:
for row in df['image_id']:
    print(row)

000011648.jpg
000011630.jpg
000011650.jpg
000011633.jpg
000011656.jpg
000011631.jpg
000011645.jpg
000011627.jpg
000011653.jpg
000011640.jpg
000011641.jpg
000011651.jpg
000011634.jpg
000011652.jpg
000011643.jpg
000011657.jpg
000011646.jpg
000011655.jpg
000011642.jpg
000011636.jpg
000011637.jpg
000011635.jpg
000011647.jpg
000011628.jpg
000011632.jpg
000011644.jpg
000011629.jpg
000011638.jpg
000011654.jpg
000011639.jpg
000011649.jpg
000012428.jpg
000012434.jpg
000012430.jpg
000012413.jpg
000012417.jpg
000012412.jpg
000012431.jpg
000012436.jpg
000012424.jpg
000012427.jpg
000012423.jpg
000012435.jpg
000012426.jpg
000012414.jpg
000012420.jpg
000012416.jpg
000012421.jpg
000012437.jpg
000012419.jpg
000012432.jpg
000012425.jpg
000012415.jpg
000012429.jpg
000012418.jpg
000012422.jpg
000012433.jpg
000033492.jpg
000033496.jpg
000033501.jpg
000033502.jpg
000033498.jpg
000033490.jpg
000033493.jpg
000033503.jpg
000033495.jpg
000033491.jpg
000033504.jpg
000033500.jpg
000033497.jpg
000033494.jpg
000033

In [10]:
data_folder = "/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"

## Resizing the images to 256 * 256 or 512 * 512

In [11]:
# PAD = True
WIDTH = 256  # update to 256 or 512
HEIGHT = 256 # update to 256 or 512

In [12]:
# def pad_image(img):
#     w, h, c = np.shape(img)
#     if w > h:
#         pad = int((w - h) / 2)
#         img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
#     else:
#         pad = int((h - w) / 2)
#         img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
#     return img

In [13]:
def open_and_preprocess_image(row_df):
    img = cv2.imread(data_folder + 'train_images/'+str(row_df.hotel_id)+'/'+str(row_df.image_id))
#     if PAD:
#         img = pad_image(img)
    return cv2.resize(img, (WIDTH, HEIGHT))

In [14]:
# import threading

# # create a lock object
# lock = threading.Lock()

# def save_image(row_df, img):
#     output_folder = "/kaggle/working/resized_train_images/"
#     path = os.path.join(output_folder, str(row_df.hotel_id))
#     if os.path.exists(path):
#         with lock:
#             # acquire the lock before writing the file
#             cv2.imwrite(os.path.join(path,str(row_df.image_id)), img)
#     else:
#         with lock:
#             # acquire the lock before creating the directory and writing the file
#             os.makedirs(path)
#             cv2.imwrite(os.path.join(path,str(row_df.image_id)), img)

In [15]:
def save_image(row_df, img):
    output_folder = "/kaggle/working/resized_train_images_256x256/"
    path = os.path.join(output_folder, str(row_df.hotel_id))
    if os.path.exists(path):
        cv2.imwrite(os.path.join(path, str(row_df.image_id)), img)
    else:
        try:
            os.makedirs(path)
            cv2.imwrite(os.path.join(path, str(row_df.image_id)), img)
        except FileExistsError:
            # directory already exists, ignore the error
            pass

In [16]:
def process_row(row_df):
    img = open_and_preprocess_image(row_df)
    save_image(row_df, img)

### Running in parallel as dataset is large (4 threads)

In [17]:
%%time
# dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_row)(train_df.loc[i]) for i in range(0, len(train_df)))
dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_row)(df.loc[i]) for i in range(0, len(df)))

CPU times: user 58min 17s, sys: 1min 42s, total: 1h
Wall time: 17min 8s


### Saving resized images as zip (can be downloaded from kaggle)

In [18]:
!cd /kaggle/working/ && zip -qr resized_train_256x256.zip .

In [20]:
# imageee =  cv2.imread('/kaggle/working/resized_train_images/202863/000013861.jpg')
# imageee.shape

In [ ]:
[df.loc[i] for i in range(0, len(df))]

In [15]:
# image = cv2.imread('/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/../000011648.jpg')
# type(image)
# image.imshow()

In [ ]:
# !cd /kaggle/working/ & zip -jqr train.zip .
# !find . -name "*.jpg" -delete

In [ ]:
print(os.listdir("/kaggle/working/resized_train_images/"))

If you want to delete from the kaggle working directory

In [41]:
import shutil
shutil.rmtree("/kaggle/working/resized_train.zip")

NotADirectoryError: [Errno 20] Not a directory: '/kaggle/working/resized_train.zip'

### Reference 
https://github.com/shubham-shetty/Hotel-ID-Against-Human-Trafficking/blob/main/notebooks/data_preprocess.ipynb


In [ ]:
# kaggle kernels output akankshajo/hotelididentification-preprocessing -p /path/to/dest